In this notebook we show how to prepare all the input and parameter files necessary to run Diva4D.<br>
The code rely on the classes defined for 2D (module `pydiva2d`) and 4D (module `pydiva4d`). 

In [72]:
import os
import sys
import logging
import shutil
import subprocess
import datetime
import pydiva4d

# User inputs and configuration

Configure logging

In [73]:
pydiva4d.logger.handlers[0].setLevel(logging.DEBUG)

Indicate the path to the Diva installation you want to work with (ending with `diva-x.y.z`).

In [74]:
divabasedir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.6.1"

## Create the sub-directories and files

In [75]:
divadirs = pydiva4d.Diva4DDirectories(divabasedir)

In [76]:
diva2Dfiles = pydiva4d.Diva2Dfiles(divadirs.diva2d)

In [77]:
diva4Dfiles = pydiva4d.Diva4Dfiles(divadirs.diva4d)

## List of user parameters

Set the different parameters necessary for a 4D analysis.<br>
For clarity, they are separated by input files.

### List of ODV spreadsheet files

An example data file is distributed with the Diva code (*Example4D* directory): 

In [78]:
datasource = pydiva4d.Datasource(os.path.join(divadirs.divaexample, "blacksea_data_CTD.txt"))

2018-09-18 14:48:58,257 - pydiva4d - WARNING - Argument converted from string to list


### Topography

In [79]:
topofile = os.path.join(divadirs.divaexample, "input/topo.grd")
topoinfofile = os.path.join(divadirs.divaexample, "input/TopoInfo.dat")

### Parameters for the advection constraint and the reference field

In [80]:
advection_flag = 0               # indicates if advection is activated
ref_flag = 0                     # indicates if there is a reference field
var_year_code = '00000000'       # variable year code
var_month_code = '0000'          # variable month code

In [81]:
constandrefe = pydiva4d.Constandrefe(advection_flag, ref_flag, var_year_code, var_month_code)

### Parameters for the 4D analysis

In [82]:
extraction_flag = 1              # Data extraction: 1 do it, 0 do nothing, -1 press coord, -10 pressure+Saunders
coast_flag = 1                   # Boundary lines and coastlines generation: 0 nothing, 1: contours, 2: UV, 3: 1+2
clean_flag = 0                   # Cleaning data on mesh: 1, 2: RL, 3: both, 4: 1 + outliers elimination, 5: =4+2
min_datanum = 0                  # Minimal number of data in a layer. If less, uses data from any month.
param_flag = -30                 # Parameters estimation and vertical filtering
min_l, max_l = 1.4, 10           # Minimal/maximal correlation length during optimization
min_snr, max_snr = 0.1, 5.0      # Minimal/maximal signal-to-noise ratio during optimization
analysisref_flag = 1             # Creation of reference field: 2 do reference, 1 do analysis and 0 do nothing
lower_level, upper_level = 1, 4  # Depth lower/upper level numbers
netcdf4d_flag = 1                # 4D netcdf files: 0 (no generation) or 1
gnuplot_flag = 0                 # Gnuplots generation: 0 (no generation) or 1
detrend_groupnum = 0             # Data detrending: number of groups, 0 if no detrending.

In [83]:
driver = pydiva4d.Driver(extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l,\
                               max_l, min_snr, max_snr, analysisref_flag, lower_level,\
                               upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum)

### Parameters for the netCDF file

In [84]:
# Title string for 3D NetCDF file
ncdf_title = '4D Test on the Black Sea using Jupyter notebook'
# Reference time for data (ie: days since since 1900-01-01), if not climatological data
ncdf_reftime = 'months since since xxxx-01-01'
# Time value (that reprsents the data set), if not climatological data
ncdf_timeval = 1200
# Cell_methode string
ncdf_cellmethod = 'time: mean (this month data from all years)'
# Institution name: where the dataset was produced.
ncdf_institution = 'University of Liege, GeoHydrodynamics and Environment Research'
# Production group and e-mail
ncdf_groupemail = 'Diva group. E-mails : a.barth@ulg.ac.be ; swatelet@ulg.ac.be ; ctroupin@ulg.ac.be'
# Source (observation, radiosonde, database, model-generated data,...)
ncdf_source = 'data_from various sources for diva software testing work'
ncdf_comment = 'No comment'
ncdf_authoremail = 'Name@institution'
ncdf_acknowlegment = 'No acknowledgement'

In [85]:
netcdfinfo = pydiva4d.Ncdfinfo(ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
                                     ncdf_institution, ncdf_groupemail, ncdf_source,\
                                     ncdf_comment, ncdf_authoremail, ncdf_acknowlegment)

### Parameters for the output grid

In [86]:
xmin, ymin = 27.0, 40.0        # longitute and latitude of the lower-left corner of the domain
nx, ny = 151, 76               # number of cells in the zonal and meridional directions
dx, dy = 0.1, 0.1              # spatial steps in the zonal and meridional directions

### Parameters for the analysis

In [87]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -99.
iCoordChange = 2
iSpec = 11
iReg = 2

In [88]:
parameters2D = pydiva4d.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

## Others: monthlist, QC flags, variables, years, contour levels

In [89]:
months = pydiva4d.Monthlist(['0101', '0202', '0303'])       # list of strings representing the periods (months)
qflags = pydiva4d.Qflist([0, 1])                            # list of integers representing the accepted flags
variables = pydiva4d.Varlist(['Temperature', 'Salinity'])   # list of strings representing the variables to extract
years = pydiva4d.Yearlist(['19002009'])                     # list of strings representing the perior (years)
contours = pydiva4d.Contourdepth([30., 20., 10., 0.])       # list of floats representing the vertical levels

# Create the new input files

Now we have set all the parameters, we can write the information into the corresponding `Diva` files.<br>
All the objects have a *write_to* method that is used to write the information to a file.

## Write the different input files

In [90]:
datasource.write_to(diva4Dfiles.datasource)
constandrefe.write_to(diva4Dfiles.constandrefe)
driver.write_to(diva4Dfiles.driver)
months.write_to(diva4Dfiles.monthlist)
qflags.write_to(diva4Dfiles.qflist)
variables.write_to(diva4Dfiles.varlist)
years.write_to(diva4Dfiles.yearlist)
netcdfinfo.write_to(diva4Dfiles.ncdfinfo)
parameters2D.write_to(diva4Dfiles.param)
contours.write_to(diva4Dfiles.contourdepth)

## Copy the topography files

In [100]:
shutil.copy2(topofile, diva4Dfiles.topography)
shutil.copy2(topoinfofile, diva4Dfiles.topoinfo)

'/home/ctroupin/Software/DIVA/DIVA-diva-4.6.1/JRA4/Climatology/./input/TopoInfo.dat'

In [101]:
topofile

'/home/ctroupin/Software/DIVA/DIVA-diva-4.6.1/Example4D/input/topo.grd'

# Run divadoall (4D analysis)

We call the main script `divadoall` to perform the 4D analysis.<br>
The time for the analysis depends on the number of variables, levels, periods, etc, so can expect it to run for a few minutes with the default values set in this example.

In [102]:
pydiva4d.logger.info("Starting to run 'divadoall'")
divadoall_script = './divadoall'
env = os.environ.copy()
env["PATH"] = ".:" + env["PATH"]

p = subprocess.Popen(divadoall_script, stdout=subprocess.PIPE, cwd=divadirs.diva4d, env=env)
out = p.stdout.read()
pydiva4d.logger.info("Finished 'divadoall'")

Finally we write the `Diva` execution log into the same log file:

In [103]:
pydiva4d.logger.info("Diva execution log:")
with open(pydiva4d.logfile, 'w') as f:
    f.write(str(out).replace('\\n', '\n'))

In [104]:
pydiva4d.logger.info("Finished the 'divadoall' run")

In [105]:
pydiva4d.logfile

'./logs/Diva_2018-09-18_14:14:17.log'

## Data visualisation

### ncview

If you have the [ncview](http://meteora.ucsd.edu/%7Epierce/ncview_home_page.html) tool installed, you can have a quick look to the different layers, periods and variables to ensure the `Diva` execution was fine.<br>
The name of the output file is easily obtained from the variable names and the period.

In [106]:
Temperature4Dfile = os.path.join(divadirs.diva4doutput3d, "Temperature.19002009.4Danl.nc")

In [107]:
process = subprocess.Popen(['/usr/bin/ncview', Temperature4Dfile], stdout=subprocess.PIPE)

## matplotlib

In the notebooks [plot_diva4D_results](plot_diva4D_results.ipynb) and [plot_diva4D_results_basemap.ipynb](plot_diva4D_results_basemap.ipynb) we show how to use [matplotlib](http://matplotlib.org/) and [Basemap](http://matplotlib.org/basemap/api/basemap_api.html) to produce figures from the 4D netCDF file.